In [1]:
import re
import pandas as pd;
from math import log2

In [2]:
class Node:
    """ Node class for a decision tree. """
    def __init__(self, names):
        self.names = names

    def classify(x):
        """ Handled by the subclasses. """
        return None

    def dump(self, indent):
        """ Handled by the subclasses. """
        return None


class Leaf(Node):
    def __init__(self, names, value):
        Node.__init__(self, names)
        self.value = value

    def classify(self, x):
        return self.value

    def dump(self, indent):
        print(' %d' % self.value)


class Split(Node):
    def __init__(self, names, var, left, right):
        Node.__init__(self, names)
        self.var = var
        self.left = left
        self.right = right

    def classify(self, x):
        if x[self.var] == 0:
            return self.left.classify(x)
        else:
            return self.right.classify(x)
      
    def dump(self, indent):
        if indent > 0:
            print('')
        for i in range(0, indent):
            print('| ', end='')
        print('%s = 0 :' % self.names[self.var],end='')
        self.left.dump(indent+1)
        for i in range(0, indent):
            print('| ', end='')
        print('%s = 1 :' % self.names[self.var],end='')
        self.right.dump(indent+1)


Helper function computes entropy of Bernoulli distribution with parameter p

In [3]:
def entropy(p):
    pr,tsamples  = [], p.groupby(p).count()
    for i in tsamples:
        pr.append(i/len(p))
    var_entropy = 0
    for i in pr:
        var_entropy += -(i*log2(i))
    return var_entropy    

Compute information gain for a particular split, given the counts 

py_pxi : number of occurences of y=1 with x_i=1 for all i=1 to n

pxi : number of occurrences of x_i=1

py : number of ocurrences of y=1


In [4]:
def infogain(data, varnames):
    df = pd.DataFrame(data)
    target = df[df.columns[-1]]
    ent, tig =  entropy(target), {}
    for i in range(len(df.columns)-1):
        val_cnt, temp = {}, 0
        feature = df[i]
        for a in feature.groupby(feature).groups.values():
            ss = []
            for b in feature.iloc[a].groupby(target).count():
                ss.append(b)
            val_cnt[temp]  = ss
            temp += 1            
        fcount = val_cnt
        total = len(df[i])        
        m_ent = 0        
        for key, value in fcount.items():
            pr = []
            for a in value:
                pr.append(a/sum(value))
            e = 0
            for x in pr:
                e += -(x * log2(x))
            m_ent += ((sum(value)/total) * e)
        gain = ent - m_ent
        tig[varnames[i]] = gain
        
    return tig

OTHER SUGGESTED HELPER FUNCTIONS:

-collect counts for each variable value with each class label

-find the best variable to split on, according to mutual information

-partition data based on a given variable	
	


In [5]:
# Load data from a file
def read_data(filename):
    f = open(filename, 'r')
    p = re.compile(',')
    data = []
    header = f.readline().strip()
    varnames = p.split(header)
    namehash = {}
    for l in f:
        data.append([int(x) for x in p.split(l.strip())])
    return (data, varnames)


Build tree in a top-down manner, selecting splits until we hit a pure leaf or all splits look bad.


In [6]:
def build_tree(data, varnames):
    df = pd.DataFrame(data)
    info_gain = infogain(data, varnames)
    max_gain = max(info_gain, key=info_gain.get)
    
    if info_gain[max_gain] == 0.0:
        return Leaf(varnames,df[df.columns[-1]].unique()[0])
    
    if max_gain in varnames:
           gain = varnames.index(max_gain) 
            
    tc = df[gain].unique()
    if len(tc) == 1:
        return Leaf(varnames,df[gain].unique()[0])
    else:
        left_tree = df[df[gain]==0]
        left_tree.reset_index(inplace=True,drop=True)
        
        right_tree = df[df[gain] == 1]
        right_tree.reset_index(inplace=True,drop=True)
        
        root = Split(varnames, gain , build_tree(left_tree,varnames), build_tree(right_tree,varnames))
    return root


Here we load data.
Each example is a list of attribute values, where the last element in the list is the class value.


In [7]:
(train, varnames) = read_data('agaricuslepiotatrain1.csv')
(test, testvarnames) = read_data('agaricuslepiotatest1.csv') 

In [8]:
pd.DataFrame(train)

0    1    2    3    4    5    6    7    8    9    ...  113  114  115  \
0       0    0    1    0    0    0    0    0    0    1  ...    0    0    0   
1       0    0    1    0    0    0    0    0    0    1  ...    0    0    1   
2       1    0    0    0    0    0    0    0    0    1  ...    0    0    0   
3       0    0    1    0    0    0    0    0    1    0  ...    0    0    0   
4       0    0    1    0    0    0    0    0    0    1  ...    0    0    1   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
5995    0    0    1    0    0    0    0    0    1    0  ...    1    0    0   
5996    0    0    0    0    1    0    0    0    1    0  ...    0    0    0   
5997    0    0    1    0    0    0    0    0    1    0  ...    0    1    0   
5998    0    0    0    1    0    0    0    0    0    1  ...    1    0    0   
5999    0    0    0    1    0    0    0    0    1    0  ...    0    1    1   

      116  117  118  119  120  121  122  
0       0    0    0    1    0    0    1  
1       0    0    0    0    0    0    0  
2       0    1    0    0    0    0    0  
3       0    0    0    1    0    0    1  
4       0    0    0    0    0    0    0  
...   ...  ...  ...  ...  ...  ...  ...  
5995    0    0    0    0    0    1    1  
5996    0    0    0    0    1    0    0  
5997    0    0    0    0    0    1    0  
5998    0    0    0    1    0    0    1  
5999    0    0    0    0    0    0    1  

[6000 rows x 123 columns]

In [9]:
varnames

['cap-shape-bell',
 'cap-shape-conical',
 'cap-shape-convex',
 'cap-shape-flat',
 'cap-shape-knobbed',
 'cap-shape-sunken',
 'cap-surface-fibrous',
 'cap-surface-grooves',
 'cap-surface-scaly',
 'cap-surface-smooth',
 'cap-color-brown',
 'cap-color-buff',
 'cap-color-cinnamon',
 'cap-color-gray',
 'cap-color-green',
 'cap-color-pink',
 'cap-color-purple',
 'cap-color-red',
 'cap-color-white',
 'cap-color-yellow',
 'bruises?-bruises',
 'odor-almond',
 'odor-anise',
 'odor-creosote',
 'odor-fishy',
 'odor-foul',
 'odor-musty',
 'odor-none',
 'odor-pungent',
 'odor-spicy',
 'gill-attachment-attached',
 'gill-attachment-descending',
 'gill-attachment-free',
 'gill-attachment-notched',
 'gill-spacing-close',
 'gill-spacing-crowded',
 'gill-spacing-distant',
 'gill-size-broad',
 'gill-color-black',
 'gill-color-brown',
 'gill-color-buff',
 'gill-color-chocolate',
 'gill-color-gray',
 'gill-color-green',
 'gill-color-orange',
 'gill-color-pink',
 'gill-color-purple',
 'gill-color-red',
 'gill

Build the decision tree

In [10]:
root = build_tree(train, varnames)

In [11]:
root.dump(0)

odor-foul = 0 :
| gill-size-broad = 0 :
| | odor-none = 0 :
| | | gill-spacing-close = 0 :
| | | | bruises?-bruises = 0 : 1
| | | | bruises?-bruises = 1 : 0
| | | gill-spacing-close = 1 : 1
| | odor-none = 1 :
| | | stalk-surface-above-ring-silky = 0 :
| | | | bruises?-bruises = 0 : 0
| | | | bruises?-bruises = 1 : 1
| | | stalk-surface-above-ring-silky = 1 : 1
| gill-size-broad = 1 :
| | spore-print-color-green = 0 : 0
| | spore-print-color-green = 1 : 1
odor-foul = 1 : 1


Calcuating the accuracy

In [12]:
def accuracy(data):
    correct = 0
    # The position of the class label is the last element in the list.
    yi = len(train[0]) - 1
    for x in train:
    # Classification is done recursively by the node class.
    # This should work as-is.
        pred = root.classify(x)
        if pred == x[yi]:
            correct += 1
        acc = float(correct)/len(train)
    return acc;
   

In [13]:
 print("Train Accuracy: {}".format(accuracy(train)))

Train Accuracy: 1.0


In [14]:
 print("Test Accuracy: {}".format(accuracy(test)))

Test Accuracy: 1.0
